# Cropping the first quadrant video

In [13]:
import cv2

In [14]:
video_path = '/Users/vaibhavmudgal/driving_7-4-8-1-9-2-5-6-9174819256.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video capture object is opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
else:
    # Get the dimensions of the video
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Video dimensions: {width}x{height}")

    cap.release()

Video dimensions: 1440x1076


In [17]:
import cv2

# Load the video
video_path = '/Users/vaibhavmudgal/driving_7-4-8-1-9-2-5-6-9174819256.mp4'
cap = cv2.VideoCapture(video_path)

# Get video dimensions
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the coordinates for cropping (first quadrant)
start_x = 0
end_x = frame_width // 2
start_y = 0
end_y = frame_height // 2

# Create a VideoWriter to save the cropped video
output_path = 'output_cropped_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, 30, (end_x - start_x, end_y - start_y))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Crop the frame
    cropped_frame = frame[start_y:end_y, start_x:end_x]
    
    # Write the cropped frame to the output video
    out.write(cropped_frame)
    
    # Display the cropped frame (optional)
    cv2.imshow('Cropped Video', cropped_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything when done
cap.release()
out.release()
cv2.destroyAllWindows()


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


# Adding boxes for central cropping

In [2]:
import cv2

# Path to the input video file
video_path = 'driving_7-4-8-1-9-2-5-6-9174819256-2.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get the video frame dimensions
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Calculate box sizes based on percentages
box_sizes = [0.3, 0.5, 0.7, 0.9]

# Define codec and output video file
output_path = 'output_video_with_boxes_2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can change the codec as needed
out = cv2.VideoWriter(output_path, fourcc, 30.0, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Draw bounding boxes with dotted lines and text labels
    for size in box_sizes:
        box_width = int(frame_width * size)
        box_height = int(frame_height * size)
        x_start = (frame_width - box_width) // 2
        y_start = (frame_height - box_height) // 2
        x_end = x_start + box_width
        y_end = y_start + box_height
        
        # Draw dotted rectangle
        dot_interval = 10  # Adjust the interval as needed
        for i in range(x_start, x_end, dot_interval):
            cv2.line(frame, (i, y_start), (i + dot_interval // 2, y_start), (0, 255, 0), 1)
            cv2.line(frame, (i, y_end), (i + dot_interval // 2, y_end), (0, 255, 0), 1)
        for i in range(y_start, y_end, dot_interval):
            cv2.line(frame, (x_start, i), (x_start, i + dot_interval // 2), (0, 255, 0), 1)
            cv2.line(frame, (x_end, i), (x_end, i + dot_interval // 2), (0, 255, 0), 1)
        
        # Add text label
        label = f"{int(size * 100)}%"  # Convert size to percentage
        label_x = x_start
        label_y = y_start - 10  # Adjust the position as needed
        cv2.putText(frame, label, (label_x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    # Write the modified frame to the output video
    out.write(frame)
    
#     # Display the frame with bounding boxes
#     cv2.imshow('Video with Bounding Boxes', frame)
    
    # Exit loop if 'q' key is pressed
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release video capture, release the output video, and close windows
cap.release()
out.release()
cv2.destroyAllWindows()


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


# Morphing

In [6]:
import cv2
import numpy as np

# Paths to the input videos
video_path_1 = '/Users/vaibhavmudgal/driving_7-4-8-1-9-2-5-6-9174819256.mp4'
video_path_2 = 'output_video_with_boxes.mp4'

# Open the video files
cap_1 = cv2.VideoCapture(video_path_1)
cap_2 = cv2.VideoCapture(video_path_2)

# Get the video frame dimensions
frame_width = int(cap_1.get(3))
frame_height = int(cap_1.get(4))

# Define the region of interest (first quadrant)
roi_width = frame_width // 2
roi_height = frame_height // 2

# Define codec and output video file
output_path = 'output_morphed_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can change the codec as needed
out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame_width, frame_height))

while cap_1.isOpened() and cap_2.isOpened():
    ret_1, frame_1 = cap_1.read()
    ret_2, frame_2 = cap_2.read()
    
    if not ret_1 or not ret_2:
        break
    
    # Resize the frames to match the dimensions of the first video
    frame_2 = cv2.resize(frame_2, (roi_width, roi_height))
    
    # Create a blank canvas
    morphed_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
    
    # Fill the first quadrant with frame_2
    morphed_frame[:roi_height, :roi_width] = frame_2
    
    # Fill the remaining quadrants with frame_1
    morphed_frame[roi_height:, :roi_width] = frame_1[roi_height:, :roi_width]
    morphed_frame[:, roi_width:] = frame_1[:, roi_width:]
    
    # Write the morphed frame to the output video
    out.write(morphed_frame)
    
#     # Display the morphed frame
#     cv2.imshow('Morphed Video', morphed_frame)
    
    # Exit loop if 'q' key is pressed
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release video captures, release the output video, and close windows
cap_1.release()
cap_2.release()
out.release()
cv2.destroyAllWindows()



OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
